In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

# Directory to save generated images
os.makedirs("pattern_data", exist_ok=True)

def generate_shape(shape, filename):
    """Generate images of geometric shapes."""
    fig, ax = plt.subplots(figsize=(2, 2))
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)

    if shape == "circle":
        radius = random.randint(10, 30)
        x, y = random.randint(30, 70), random.randint(30, 70)
        circle = plt.Circle((x, y), radius, color='blue')
        ax.add_artist(circle)
    elif shape == "square":
        size = random.randint(10, 30)
        x, y = random.randint(30, 70), random.randint(30, 70)
        square = plt.Rectangle((x, y), size, size, color='green')
        ax.add_artist(square)
    elif shape == "triangle":
        x = random.randint(30, 70)
        y = random.randint(30, 70)
        size = random.randint(10, 30)
        triangle = plt.Polygon([(x, y), (x+size, y), (x+size/2, y+size)], color='red')
        ax.add_artist(triangle)
    
    ax.axis('off')
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close()

# Generate 100 images for each shape
for shape in ["circle", "square", "triangle"]:
    for i in range(100):
        filename = f"pattern_data/{shape}_{i}.png"
        generate_shape(shape, filename)

print("Shape images generated!")


In [ ]:
import cv2
import glob

def preprocess_images(folder):
    """Convert images to grayscale and resize them."""
    images = []
    labels = []
    for filepath in glob.glob(f"{folder}/*.png"):
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
        img_resized = cv2.resize(img, (50, 50))  # Resize to 50x50 pixels
        images.append(img_resized)

        # Extract the label from filename
        label = filepath.split("/")[-1].split("_")[0]
        labels.append(label)
    
    return np.array(images), np.array(labels)

images, labels = preprocess_images("pattern_data")
print("Preprocessing complete!")
print(f"Number of samples: {len(images)}")


In [3]:
from sklearn.preprocessing import LabelEncoder

# Flatten images
features = images.reshape(len(images), -1)

# Encode labels (circle -> 0, square -> 1, triangle -> 2)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

print("Features and labels prepared!")
print(f"Feature shape: {features.shape}")
print(f"Label mapping: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")


Features and labels prepared!
Feature shape: (300, 2500)
Label mapping: {np.str_('pattern'): np.int64(0)}


In [4]:
from sklearn.cluster import KMeans

# Train KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(features)

# Map cluster labels back to shape names
cluster_mapping = {}
for i, label in enumerate(kmeans.labels_[:10]):  # Assume the first 10 are well-clustered
    cluster_mapping[label] = labels[i]

print("Clustering complete!")
print("Cluster to shape mapping:", cluster_mapping)


Clustering complete!
Cluster to shape mapping: {np.int32(2): np.str_('pattern'), np.int32(1): np.str_('pattern')}


In [5]:
def predict_shape(image_path):
    """Predict shape for a new image."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (50, 50)).flatten().reshape(1, -1)
    
    cluster = kmeans.predict(img_resized)[0]
    predicted_shape = cluster_mapping.get(cluster, "Unknown")
    
    return predicted_shape

# Test with a new image
test_image = "pattern_data/circle_0.png"  # Replace with any image path
predicted_shape = predict_shape(test_image)
print(f"Predicted shape: {predicted_shape}") 


Predicted shape: pattern


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Flatten images
features = images.reshape(len(images), -1)

# Normalize pixel values (scale between 0 and 1)
features = features / 255.0

# Encode labels (circle -> 0, square -> 1, triangle -> 2)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

print("Data prepared for Scikit-learn!")
print(f"Training data shape: {X_train.shape}")


Data prepared for Scikit-learn!
Training data shape: (240, 2500)


In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create and train the neural network
mlp = MLPClassifier(hidden_layer_sizes=(256, 128, 64), activation='relu', max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Evaluate on the test set
y_pred = mlp.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Test Accuracy: 100.00%

Classification Report:
              precision    recall  f1-score   support

     pattern       1.00      1.00      1.00        60

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



In [8]:
def predict_shape(image_path):
    """Predict shape for a new image using the trained model."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (50, 50)).flatten() / 255.0  # Normalize and flatten
    prediction = mlp.predict([img_resized])
    predicted_shape = label_encoder.inverse_transform(prediction)[0]
    return predicted_shape

# Test with a new image
test_image = "pattern_data/circle_0.png"  # Replace with any image path
predicted_shape = predict_shape(test_image)
print(f"Predicted shape: {predicted_shape}")


Predicted shape: pattern


In [ ]:
import cv2
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder

# Simulated training process (reuse this from earlier)
# Assume 'mlp' is your trained MLPClassifier and 'label_encoder' is ready

# Example function to predict the shape from an image
def predict_shape(image_path):
    """Predict the geometric shape in the image."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return "Invalid image path or file not found."

    # Preprocess: Resize, normalize, and flatten
    img_resized = cv2.resize(img, (50, 50)).flatten() / 255.0
    prediction = mlp.predict([img_resized])
    predicted_shape = label_encoder.inverse_transform(prediction)[0]
    return predicted_shape

# Q&A Loop
def pattern_recognition_qa():
    print("Welcome to the Pattern Recognition Q&A!")
    print("Type 'exit' to quit.")
    while True:
        question = input("\nAsk a question (e.g., 'What shape is in image path <path>?'): ").strip()
        if question.lower() == 'exit':
            print("Goodbye!")
            break

        # Check if the question follows the format for shape prediction
        if "What shape is in image path" in question:
            try:
                # Extract the image path from the question
                image_path = question.split("image path")[-1].strip()
                result = predict_shape(image_path)
                print(f"Answer: The shape in the image is {result}.")
            except Exception as e:
                print(f"Error: {e}")
        else:
            print("Sorry, I can only answer questions about shapes in image paths.")

# Start the Q&A loop
pattern_recognition_qa()


Welcome to the Pattern Recognition Q&A!
Type 'exit' to quit.



Ask a question (e.g., 'What shape is in image path <path>?'):  what shape is in image path


Sorry, I can only answer questions about shapes in image paths.



Ask a question (e.g., 'What shape is in image path <path>?'):  What shape is in image path pattern_data/circle_0.png?


Answer: The shape in the image is Invalid image path or file not found..
